# CASO DE ESTUDIO 3 - PREDICCION DEL PRECIO DEL ORO
Predicción del precio del oro con redes LSTM

## 1. Business case discovery

### 1.2 Contexto

Optimizar inveriones en metales preciosos. El oro es afectado por **inflacion, tasas de interes, crisis geopoliticas, etc**. 

Se busca analisis de datos historicos del precio del oro *(XAU/USD)* y variables macroeconomicas para predecir su valor.

### 1.3 Objetivo del proyecto

Construir una LSTM que prediga el precio del oro para los proximos **5 dias habiles**. Se usaran:
- Datos historicos diarios (2010-2024)
- Indicadores macro: Indice dolar (DXY), tasas de interes FED, inflacion, etc
- Metrica de exito: **RMSE < 2.5%**

# 2. Preprocesamiento de datos

## 2.0.5 Importacion de librerias necesarias

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyploy as plt
import seaborn as sns
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.model_selection import KFold

## 2.1 Origen de datos
- Precio: Yahoo Finance (`GC-F`)
- Datos macro: Yahoo Finance

In [ ]:
# Todos los datos necesarios se pueden obtener con estos tickers públicos:
import yfinance as yf

# 1. Precio del oro (futuros)
df_oro = yf.download("GC=F", start="2010-01-01")

# 2. Índice Dólar (DXY)
df_dxy = yf.download("DX-Y.NYB", start="2010-01-01")  # Ticker correcto para DXY

# 3. Bonos 10 años (^TNX como proxy de tasas de interés)
df_bonos = yf.download("^TNX", start="2010-01-01")

# 4. Inflación (usar ETF TIP como proxy)
df_inflacion = yf.download("TIP", start="2010-01-01")  # ETF de bonos indexados a inflación

df_oro.head()
df_dxy.head()
df_bonos.head()
df_inflacion.head()

In [ ]:
dxy = df_dxy['Close']
bonos = df_bonos['Close']
tip = df_inflacion['Close']

df = pd.concat([oro, dxy, bonos, tip], axis=1)
df.columns = ["Oro", "DXY", "Bonos_10y", "TIP"]


## 2.2 Feature Engineering
- **Ratio Oro/Dolar**: segun el USD (`Precio Oro / DXY`)
- **Inflacion implicita**: Variacion porcentual mensual del ETF TIP (proxy para CPI)
- **Tasa real aproximada**: Rendimiento bonos 10 años (^TNX - Inflacion implicita)

In [ ]:
df['Ratio_Oro_Dolar'] = df['Oro'] / df['DXY']

llllllll # Para que de error antes
# TODO
df['Infla_implicita'] = df[''] / df['']

df['Tasa_real_aprox'] = df['Bonos_10y'] - df['Infla_implicita']

df.head()

Variables técnicas (complementarias):

- SMA de 200 días (identificar tendencias largas)
- Bandas de Bollinger (volatilidad histórica)

In [ ]:
# ?

## 2.4 Ventana temporal

Estructurar los datos en secuencias de entrada-salida para entrenar la LSTM:

- Look-back (ventana histórica): 90 días de datos (precio oro, DXY, inflación implícita, ratio oro/dólar, volatilidad) como entrada.
- Forecast horizon (horizonte de predicción): 5 días futuros de precios de cierre del oro como salida.
- Normalización robusta: Usar RobustScaler para manejar outliers en crisis económicas.
- Reformateo: Transformar los datos en tensores 3D (muestras, pasos temporales, características) usando TimeseriesGenerator de Keras.

In [ ]:
look_back = 90  # 3 meses bursátiles
horizon = 5     # Predecir 5 días futuros

data_trimmed = data_scaled[:len(data_scaled)-horizon]

generator = TimeseriesGenerator(
    data_scaled,
    targets=data_trimmed[:, 0],  # Predecir solo el precio del oro (columna 0)
    length=look_back,
    batch_size=32,
    sampling_rate=1,
    stride=1,
    shuffle=True
)

#Ejemplo de secuencia resultante
X_sample, y_sample = generator[0]
print(f"Input shape: {X_sample.shape}")  # (batch, 90 días, 5 features)
print(f"Target shape: {y_sample.shape}") # (batch, 5 días a predecir)

## 2.5 Division de datos

Dividir cronológicamente para preservar la integridad temporal:

- **Entrenamiento (2010-2018)**: Datos con crisis históricas relevantes.
- **Validación (2019-2020)**: Período COVID-19 para probar resiliencia ante shocks.
- **Test (2021-2024)**: Datos recientes con alta inflación y tensión geopolítica.

In [ ]:
train = data.loc['2010':'2018']
val = data.loc['2019':'2020']
test = data.loc['2021':]

print(train.shape)
print(val.shape)
print(test.shape)

> Tengo que separar en features y labels para el entrenamiento y validacion

In [ ]:
# 

# 3. Model planning

## 3.1 Definicion del problema

El proyecto se enmarca como un problema de regresión multivariante secuencial, donde múltiples características (precio del oro, índice DXY, inflación, tasas de interés o volumen) se usan para predecir una secuencia futura. La LSTM es ideal por su capacidad para recordar patrones a largo plazo y gestionar dependencias temporales complejas, como ciclos macroeconómicos, crisis geopolíticas, y correlación histórica oro-dólar.

## 3.2 Arquitectura de la red

- **Capa LSTM *(192 unidades)***: Captura relaciones complejas en secuencias largas (90 días), incluyendo patrones de demanda física (ej: compras centrales bancarias).
- **Dropout *(35%)***: Aleatoriza la desactivación de neuronas para contrarrestar sobreajuste en datos con eventos black-swan (ej: crisis 2008, COVID-19).
- **Capa LSTM *(96 unidades)***: Refina los patrones aprendidos, enfocándose en dependencias macroeconómicas a mediano plazo (ej: impacto de reuniones de la FED).
- **Capa Densa *(5 neuronas)***: Genera predicciones para los 5 días futuros, usando activación lineal para regresión.

## 3.3 Función de Pérdida y Optimizador

- **Función de pérdida**: Error cuadrático medio (***MSE***) con ponderación exponencial (da 2x más peso a errores en períodos de alta volatilidad).
- **Optimizador**: ***Adam*** con tasa de aprendizaje variable (lr=0.001 inicial, reduciendo a 0.0001 después de 50 épocas).
- **Métricas adicionales**:
    - ***MAE***: Error absoluto medio
    - ***Directional Accuracy***: Precisión direccional personalizada para oro (umbral de ±0.8% para considerar movimiento significativo)

### 3.3.1 Creacion de la tasa de aprendizaje variable


In [ ]:
# Crear un callback para cambiar el learning rate después de 50 épocas
def scheduler(epoch, lr):
    if epoch == 50:
        return 0.0001
    else:
        return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

## 3.4 Estrategias Contra el Sobreajuste (OPCIONAL si lo ves necesario)

1. **Early Stopping**: Detener entrenamiento si pérdida en validación no mejora en 15 épocas
2. **Regularización L2 (λ=0.001)**: Aplicada solo a capas LSTM para evitar sobre-énfasis en features macroeconómicas.
3. **Aumentación de datos**: **<u>(OPCIONAL)</u>**
    - **Ruido gaussiano (σ=0.2)**: en secuencias de entrenamiento
    - **Time Warping**: Deformaciones temporales controladas (±5 días) para simular ciclos económicos acelerados/retrasados

# 4. Model building

## 4.1 Creacion y complicacion del modelo

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(90, 6)),
        tf.keras.layers.LSTM(192),
        tf.keras.layers.Dropout(0.35),

        tf.keras.layers.LSTM(96),
        tf.keras.layers.Dense(5) # Prediccion para 5 dias
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.MSE(),
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )
    return model

## 4.2 Entrenamiento del modelo

In [ ]:
if tf.config.list_physical_devices("GPU"):
    print("Usando GPU")
    with tf.device("GPU:0"):
        model = create_model()
        
        # TODO
        model.fit(
            x_train, y_train,
            epochs=1000,
            validation_data=(x_val, y_val),
            callbacks=[lr_callback]
        )
else
    print("Sin GPU")

In [ ]:
# TODO
results = model.evaluate(x_test, y_test, verbose=0)

metric_names = ['Loss', '', ]

for name, result in zip(metric_names, results):
    if name=='Loss':
        print(f"{name}: {result}")
    else:
        print(f"{name}: {result * 100:.2f}")

# 5. Presentacion de resultados

# 6. Deployment